In [ ]:

# --- CELDA 1: IMPORTACIONES (ESTÁNDAR Y DE SRC) ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
import sys
import re

warnings.filterwarnings('ignore')


project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"✅ Añadido el path raíz del proyecto: {project_root}")

# --- IMPORTACIONES MODULARIZADAS (V6) ---
try:
    
    from src.modeling.model import OptimizedProfessorModel, EnhancedProfesorDataset
    from src.modeling.training import optimized_train_epoch, optimized_validate
    from src.data_processing.utils import get_optimal_weight_iic, PESOS_DIVISIONES, get_sentiment_score
    print("✅ Módulos 'src' (Estrategia V6 - Fusión Directa) importados exitosamente.")
except ImportError as e:
    print(f"❌ ERROR CRÍTICO al importar desde src/: {e}")

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False
print(f"✅ Librerías cargadas. 🔥 CUDA disponible: {torch.cuda.is_available()}")



# --- CELDA 2: CONFIGURACIÓN, DATOS E HIPERPARÁMETROS (V6 CORREGIDO) ---


def load_data_v6(embeddings_dir, data_dir):
    
    
    embed_file = "profesores_embeddings_multilingual_robust_20250905_184407_complete.pkl" 
    embed_path = Path(embeddings_dir) / embed_file
    
    
    csv_path = Path(data_dir) / "evaluaciones_con_departamentos.csv"

    if not embed_path.exists():
        print(f"❌ ERROR FATAL: No se encontró el archivo de Embeddings: {embed_file}")
        print(f"   Por favor, copia tu .pkl funcional (de Proyect_NLP) a: {embeddings_dir}")
        print(f"   (Faltando: {embed_path})")
        return None 
    if not csv_path.exists():
        print(f"❌ ERROR FATAL: No se encontró el CSV maestro en: {csv_path}")
        return None

    print(f"📂 Cargando archivo de Embeddings: {embed_file}")
    with open(embed_path, 'rb') as f: data_pkl = pickle.load(f)

    
    embeddings = data_pkl['embeddings']
    data_dict = data_pkl['data']
    print(f"📊 Diagnóstico de carga (cruda): {len(embeddings)} muestras.")
    
    return (
        embeddings, data_dict.get('ratings', []), data_dict.get('departments', []),
        data_dict.get('divisions', None), data_dict.get('original_comments', []), 
        data_dict.get('subjects', []) 
    )

# --- Rutas  ---
EMBEDDINGS_DIR = r"src/data/Embeddings" 
DATA_DIR = r"src/data/raw/csv_completo"
MODELS_DIR = r"src/models/checkpoints" 
RESULTS_DIR = r"src/models/results" 

# --- C. Hiperparámetros  ---
EMBEDDING_DIM = 384    
HIDDEN_DIM = 256
DROPOUT = 0.5           
LEARNING_RATE = 1e-3    
EPOCHS = 30
BATCH_SIZE = 64
PATIENCE = 5 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
MODEL_NAME = f"best_model_V6_FUSION_{timestamp}.pth"
RESULTS_FILE = f"resultados_V6_FUSION_{timestamp}.json"

print(f" Configuración Cargada (V6 - LR Ajustado):")
print(f" • Device: {device}")
print(f" • Learning Rate: {LEARNING_RATE} (¡Ajustado para MLP!)")
print(f" • Dropout: {DROPOUT}")


# --- D. Cargar y Crear DataLoaders (V6) ---
train_loader = None
val_loader = None
try:
    data = load_data_v6(EMBEDDINGS_DIR, DATA_DIR)
    if data:
        full_dataset = EnhancedProfesorDataset(*data) # Pasa los 6 args de datos al Dataset V6
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=0, pin_memory=True)
        print(f"\n DataLoaders V6 creados exitosamente. Samples: {len(full_dataset)}")
    else:
        print(" Error en la carga de datos. Entrenamiento detenido.")
except Exception as e:
    print(f" ERROR durante la creación de datasets V6: {e}")


# %%
# --- CELDA 3: INICIALIZAR Y ENTRENAR (V6) ---

if train_loader and val_loader:
    # 1. Inicializar Modelo V6
    model = OptimizedProfessorModel(
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        dropout=DROPOUT
    ).to(device)

    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2, verbose=True)
    
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f" Modelo V6 (Fusión Directa) instanciado. Parámetros: {total_params:,}")

    # 2. Bucle de Entrenamiento
    print("\n" + "="*50)
    print(f" INICIANDO ENTRENAMIENTO V6 (FUSIÓN DIRECTA) | {EPOCHS} Epochs")
    print("="*50 + "\n")

    history = {'train_loss': [], 'val_loss': [], 'val_corr': [], 'val_rmse': [], 'val_sent_acc': []}
    best_val_corr = -1.0 # Empezamos negativo
    epochs_no_improve = 0
    best_epoch_stats = {}

    for epoch in range(EPOCHS):
        print(f"\n--- Epoch {epoch + 1}/{EPOCHS} ---")
        
        train_metrics = optimized_train_epoch(model, train_loader, optimizer, device, epoch)
        print(f"  TRAIN | Loss: {train_metrics['total_loss']:.4f}")

        val_metrics = optimized_validate(model, val_loader, device)
        val_loss = val_metrics['val_loss']
        val_corr = val_metrics['correlation']
        
        print(f"  VAL   | Loss: {val_loss:.4f} | Corr: {val_corr:.4f} | RMSE: {val_metrics['rmse']:.4f} | Sent Acc: {val_metrics['sentiment_accuracy']:.3f}")

        history['train_loss'].append(train_metrics['total_loss'])
        history['val_loss'].append(val_loss)
        history['val_corr'].append(val_corr)
        history['val_rmse'].append(val_metrics['rmse'])
        history['val_sent_acc'].append(val_metrics['sentiment_accuracy'])

        scheduler.step(val_loss)

        if val_corr > best_val_corr:
            print(f"   ¡Nuevo Mejor Modelo! Correlación mejoró de {best_val_corr:.4f} a {val_corr:.4f}.")
            best_val_corr = val_corr
            epochs_no_improve = 0
            
            os.makedirs(MODELS_DIR, exist_ok=True) # Asegurarse que la carpeta exista
            model_save_path = os.path.join(MODELS_DIR, MODEL_NAME)
            torch.save(model.state_dict(), model_save_path)
            
            best_epoch_stats = {
                'epoch': epoch + 1, 'best_val_corr': float(val_corr), 'val_loss': float(val_loss),
                'val_rmse': float(val_metrics['rmse']), 'val_sent_acc': float(val_metrics['sentiment_accuracy']),
                'model_path': model_save_path, # Guardamos la ruta limpia
                'hyperparameters': {'lr': LEARNING_RATE, 'dropout': DROPOUT, 'batch_size': BATCH_SIZE}
            }
        else:
            epochs_no_improve += 1
            print(f"   Sin mejora. Paciencia: {epochs_no_improve}/{PATIENCE}. (Mejor Corr: {best_val_corr:.4f})")

        if epochs_no_improve >= PATIENCE:
            print(f"\n EARLY STOPPING.")
            break

    print("\n" + "="*50)
    print(" ENTRENAMIENTO FINALIZADO")
    print(f" Mejor Correlación alcanzada: {best_epoch_stats.get('best_val_corr', -1.0):.4f} (Epoch {best_epoch_stats.get('epoch', 0)})")
    print(f" Modelo guardado en: {best_epoch_stats.get('model_path', 'N/A')}")

    results_data = {'best_epoch_metrics': best_epoch_stats, 'full_history': history}
    results_path = os.path.join(RESULTS_DIR, RESULTS_FILE)
    try:
        os.makedirs(RESULTS_DIR, exist_ok=True) # Asegurarse que la carpeta exista
        with open(results_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4, ensure_ascii=False)
        print(f" Resultados completos guardados en: {results_path}")
    except Exception as e:
        print(f" Error al guardar resultados JSON: {e}")
else:
    print("="*50)
    print(" ENTRENAMIENTO DETENIDO: Los DataLoaders no pudieron ser creados. Revisa la Celda 2 (Rutas de Datos).")

# Paso 1. Entrenamiento

# Evaluación y Resultados: Arquitectura V6 (Fusión Directa)

Presento los resultados finales de la **Arquitectura V6 (Fusión Directa)**, una evolución significativa sobre el sistema de pesos IIC original, aplicada al análisis de reseñas académicas en **Ingeniería en Informática**.

Tras descubrir que los modelos anteriores (V1-V5) fallaban en generalizar (colapsando en pruebas del mundo real) debido a datos faltantes (`Materia=NaN`) y un sobreajuste severo, se diseñó una nueva arquitectura. Este modelo V6 abandona los canales de procesamiento separados y en su lugar fusiona el embedding de texto (384 dims) directamente con *features* contextuales (Peso Depto/Div) y *features* de NLP (un Score de Sentimiento calculado), creando un vector de entrada unificado de 387 dimensiones.

---

## Configuración del Experimento (V6 Exitoso)

- **Embeddings cargados**: `profesores_embeddings_multilingual_robust_20250905_184407_complete.pkl`
- **Muestras procesadas**: `1466`
- **Muestras válidas (post-filtrado)**: `461`
- **Modelo entrenado en CUDA**: ✅ Sí
- **Parámetros totales (V6 MLP)**: **416,516** (un modelo 49% más ligero y eficiente que el V4 anterior de 815,844).
- **Hiperparámetros V6**: LR = **0.001** (Corregido para MLP), Dropout = **0.5**.

---

## 📊 Estadísticas del Dataset V6 (Fusión Directa)

El Dataset V6 crea un vector de características único por muestra, eliminando la dependencia de datos `NaN`.

- **Vector de Features**: [Embedding (384) + Peso_Depto (1) + Peso_Div (1) + Score_Sent (1)] = **387 Dims**.
- **Rating promedio**: `7.16 ± 2.34`
- **Score de Sentimiento promedio (Canal 3)**: `0.184` (Confirma que el nuevo canal de features NLP está activo y sesgado positivamente, coincidiendo con el rating promedio).

### 🔹 Distribución de Sentimientos (Basada en Rating de 461 Muestras)
- Negativo (<6.0): `135`
- Neutral (6.0-7.9): `105`
- Positivo (>=8.0): `221`

### 🔹 Ejemplo de análisis contextual (Peso de Confianza)
- **Departamento de Ciencias Computacionales**:
    - Muestras: `105`
    - Rating promedio: `6.98`
    - Peso de Confianza (Usado en Loss): `1.00`

---

## 🚀 Entrenamiento del Modelo V6 (vs. Modelo Anterior)

La corrección de la tasa de aprendizaje (de `5e-5` a `1e-3`) permitió al modelo V6 aprender, mostrando un crecimiento estable y superando al benchmark anterior.

📌 **Comparativa de Entrenamiento (Correlación en Validación):**

| Época | Modelo V4 (Benchmark 0.6571) | Modelo V6 (Nuevo Modelo 0.6700) |
|-------|------------------------------|-------------------------------|
| 1 | -0.1332 | 0.0777 |
| 5 | 0.0882 | 0.1080 |
| 10 | 0.5594 | 0.3687 |
| 15 | **0.6571** 🏆 (Sobreajustado) | 0.5637 |
| 20 | 0.6568 | 0.6164 |
| 25 | (Entrenamiento detenido) | 0.6385 |
| 30 | (Entrenamiento detenido) | **0.6700** 🏆 (Mejor Pico) |

---

## ✅ Resultados Finales: Comparativa V4 vs V6

El Modelo V6 (Fusión Directa + LR Corregido) superó al modelo anterior (Gating V4/V5 + LR Incorrecto) en todas las métricas clave.

| Métrica | Modelo V4 (Benchmark Antiguo) | Modelo V6 (¡NUEVO GANADOR!) | Mejora |
| :--- | :--- | :--- | :--- |
| **Correlación (Pico)** | 0.6571 | **0.6700** | **+ 1.96%** |
| **Sentiment Accuracy** | 64.52% | **69.90%** | **+ 5.38%** |
| **RMSE (Error)** | 0.2038 | **0.1973** | *Menor Error* |
| **MSE (Error)** | 0.0415 | **0.0389** | *Menor Error* |

*(Resultados V6 extraídos del log de entrenamiento final y el JSON de resultados)*.

## 🎯 Conclusión

El sistema V6 (Fusión Directa) ha demostrado ser superior a la arquitectura de pesos IIC (V4/V5). Al corregir la arquitectura para fusionar features directamente (Embeddings + Contexto + Sentimiento NLP) y ajustar los hiperparámetros (LR `0.001`), el modelo V6 alcanzó una correlación de **0.6700** y un **69.9%** de precisión en sentimiento.

Esto representa una mejora medible sobre el benchmark anterior (0.6571). Sin embargo, las pruebas de inferencia en el mundo real demuestran que el modelo aún sufre de **sobreajuste severo** debido al dataset extremadamente pequeño (461 muestras).

La arquitectura V6 es la correcta, pero requiere datos de entrenamiento adicionales (como los propuestos en la "Parte 3: Scraping de Facebook") para generalizar y ser funcional en producción.